# Scrape the [justETF Screener](https://www.justetf.com/en/find-etf.html)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import justetf_scraping

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

<a id="load"></a>
## Load the data

In [ ]:
etf_df = justetf_scraping.load_overview(enrich=True)

In [ ]:
etf_df.shape

In [ ]:
etf_df.head(3)

<a id="save"></a>
## Save the data

In [ ]:
etf_df.to_csv("justetf.csv", index=True)

<a id="visualize"></a>
## Visualize the data

In [ ]:
sns.histplot(etf_df, y="strategy", hue="strategy", discrete=True)

In [ ]:
sns.histplot(etf_df, y="instrument", hue="instrument", discrete=True)

In [ ]:
sns.histplot(etf_df, y="asset", hue="asset", discrete=True)

In [ ]:
sns.histplot(etf_df, y="region", hue="region", discrete=True)

In [ ]:
sns.histplot(etf_df, y="currency", hue="currency", discrete=True)

In [ ]:
sns.scatterplot(data=etf_df, x="age_in_years", y="size", alpha=0.5, linewidth=0)

The most frequent indices.

In [ ]:
index_counts = etf_df["index"].value_counts()

In [ ]:
sns.histplot(
    etf_df[etf_df["index"].isin(index_counts[index_counts > 7].index)],
    y="index",
    discrete=True,
)

<a id="search"></a>
## Search over the data

Compare accumulating MSCI World ETFs older than 2 years with fond size greater than € 50M.

In [ ]:
acc_msci_world_df = etf_df[
    (etf_df["index"] == "MSCI World")
    & (etf_df["dividends"] == "Accumulating")
    & (etf_df["age_in_years"] > 2)
    & (etf_df["size"] > 50)
]

In [ ]:
acc_msci_world_df.shape

In [ ]:
acc_msci_world_df

In [ ]:
COLUMNS_MAPPINGS = {
    "Short-term return": {
        "last_six_months": "Last 5 months",
        "last_three_months": "Last 3 months",
        "last_month": "Last month",
        "last_week": "Last week",
        "yesterday": "Yesterday",
    },
    "Mid-term return": {
        "last_five_years": "last 5 years",
        "last_three_years": "last 3 years",
        "last_year": "last year",
    },
    "Volatility": {
        "last_five_years_volatility": "Last 5 years",
        "last_three_years_volatility": "Last 3 years",
        "last_year_volatility": "Last year",
    },
    "Return/Risk": {
        "last_five_years_return_per_risk": "Last 5 years",
        "last_three_years_return_per_risk": "Last 3 years",
        "last_year_return_per_risk": "Last year",
    },
    "Drawdown": {
        "last_five_years_max_drawdown": "Last 5 years",
        "last_three_years_max_drawdown": "Last 3 years",
        "last_year_max_drawdown": "Last year",
    },
}

In [ ]:
for title, mapping in COLUMNS_MAPPINGS.items():
    df = acc_msci_world_df[mapping.keys()].reset_index().rename(columns=mapping)
    plt.figure(dpi=150)
    ax = pd.plotting.parallel_coordinates(df, "isin", colormap="tab10", alpha=0.75)
    _ = ax.set_title(title)

In [ ]:
last_four_years = justetf_scraping.overview.LAST_FOUR_YEARS

In [ ]:
df = acc_msci_world_df[list(reversed(last_four_years))]

In [ ]:
plt.figure(dpi=150)
ax = pd.plotting.parallel_coordinates(
    df.reset_index(), "isin", colormap="tab10", alpha=0.75
)
_ = ax.set_title("Return")

In [ ]:
ter = (100 - acc_msci_world_df["ter"]) / 100

In [ ]:
df = ((100 + df) / 100).mul(ter, axis=0)

In [ ]:
cum_df = df.copy()

In [ ]:
df = (df * 100 - 100).reset_index()

In [ ]:
cum_df[last_four_years[1]] *= cum_df[last_four_years[0]]
cum_df[last_four_years[2]] *= cum_df[last_four_years[1]]
cum_df[last_four_years[3]] *= cum_df[last_four_years[2]]

In [ ]:
cum_df = (cum_df * 100 - 100).reset_index()

In [ ]:
plt.figure(dpi=150)
ax = pd.plotting.parallel_coordinates(df, "isin", colormap="tab10", alpha=0.75)
_ = ax.set_title("Return excl. TER")

In [ ]:
plt.figure(dpi=150)
ax = pd.plotting.parallel_coordinates(cum_df, "isin", colormap="tab10", alpha=0.75)
_ = ax.set_title("Cumulative return since year excl. TER")